In [1]:
# Reading the excel as a dataframe:
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
df_ = pd.read_excel(r'online_retail_II.xlsx')
df_

       Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1       489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2       489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10      13085.0  United Kingdom
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00   1.25      13085.0  United Kingdom
...        ...       ...                                  ...       ...                 ...    ...          ...             ...
525456  538171     22271                 FELTCRAFT DOLL ROSIE         2 2010-12-09 20:01:00   2.95      

In [5]:
# Making a new dataframe from the data from 2010:
df = df_[df_['InvoiceDate'].dt.year == 2010].copy()
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom
45229,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0,United Kingdom
45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom
45231,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN,United Kingdom
45232,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [6]:
# Let's look at descriptive statics of the DataFrame
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,480233.0,10.436503,110.833980,-9600.00,1.00,3.0,10.00,19152.00
Price,480233.0,4.717485,152.708631,-53594.36,1.25,2.1,4.21,25111.09
Customer ID,385774.0,15352.093951,1681.836560,12346.00,13975.00,15311.0,16797.00,18287.00


In [18]:
# As we can see, there is some negative values at the columns of "Quantity" and "Price". These negatives values are caused by returned products. So, let's get rid of from returned items:
df = df[df['Price']>0]

# And check the descriptive statics again
df.describe().T


,count,mean,std,min,25%,50%,75%,max
Quantity,467608.0,11.558714,89.225409,1.000,1.00,4.0,12.00,19152.00
Price,467608.0,4.249911,66.354656,0.001,1.25,2.1,4.21,25111.09
Customer ID,376910.0,15359.531305,1680.656464,12346.000,13982.00,15311.0,16799.00,18287.00


In [19]:
# Let's check if there is any null value
df.isnull().sum()

Invoice            0
StockCode          0
Description        0
Quantity           0
InvoiceDate        0
Price              0
Customer ID    90698
Country            0
dtype: int64

In [21]:
# As we can have some null values at the column of Customer ID. So, get rid of them as well.
df.dropna(inplace=True)

#And check it again:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [23]:
# Now our DataFrame is ready for some RFM anlysis :).

# First of all we need to define anlysis date for recency values. And to do that let's look at the date of most recent order: 
print(df['InvoiceDate'].max())

# As we can see it's 2010-12-09. So, we can set an anlysis date as 2010-12-11 (2 days after most recent order date).
today_date = dt.datetime(2010, 12, 11)

2010-12-09 20:01:00


In [25]:
#Now we are ready to define RFM values:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda invoice: invoice.nunique(),
                                     'Price': 'sum'}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']
rfm

,customer_id,recency,frequency,monetary
0,12346.0,165,6,187.36
1,12347.0,3,2,162.95
2,12348.0,74,1,14.39
3,12349.0,43,3,875.34
4,12351.0,11,1,49.46
...,...,...,...,...
4194,18283.0,18,6,498.82
4195,18284.0,67,1,91.09
4196,18285.0,296,1,100.20
4197,18286.0,112,1,150.60


In [28]:
# As we can see it's a little bit hard to make a coprasion with these values. So, we need a standardization. We can standardize RFM values by creating 5 categories which will includes 5 different value range:
rfm['recency_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['frequency_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
rfm['monetary_score'] = pd.qcut(rfm['monetary'], 5, labels=[5, 4, 3, 2, 1])

rfm

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,12346.0,165,6,187.36,1,2,2
1,12347.0,3,2,162.95,5,4,3
2,12348.0,74,1,14.39,2,5,5
3,12349.0,43,3,875.34,3,3,1
4,12351.0,11,1,49.46,5,5,4
...,...,...,...,...,...,...,...
4194,18283.0,18,6,498.82,4,1,1
4195,18284.0,67,1,91.09,3,4,4
4196,18285.0,296,1,100.20,1,4,3
4197,18286.0,112,1,150.60,2,4,3


Now we could make a comprasion very easily. But a segmentation is more useful than a raw comprasion. We could make a segmentation by using the table below

<img src="1_oUKMc3gk53hccFX4YNeu1g.png" />

In [31]:
# First of all, let's make a RF Score:
rfm['RF_SCORE'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)
rfm['RF_SCORE']

0       12
1       54
2       25
3       33
4       55
        ..
4194    41
4195    34
4196    14
4197    24
4198    42
Name: RF_SCORE, Length: 4199, dtype: object

In [32]:
# Last but not least, making the segmentation:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [33]:
# Here it is, the segmentated DataFrame
rfm

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,12346.0,165,6,187.36,1,2,2,12,hibernating
1,12347.0,3,2,162.95,5,4,3,54,champions
2,12348.0,74,1,14.39,2,5,5,25,cant_loose
3,12349.0,43,3,875.34,3,3,1,33,need_attention
4,12351.0,11,1,49.46,5,5,4,55,champions
...,...,...,...,...,...,...,...,...,...
4194,18283.0,18,6,498.82,4,1,1,41,promising
4195,18284.0,67,1,91.09,3,4,4,34,loyal_customers
4196,18285.0,296,1,100.20,1,4,3,14,at_Risk
4197,18286.0,112,1,150.60,2,4,3,24,at_Risk
